El proyecto Magia es un proyecto que busca medir como responde la gente frente a una situacion novedosa y dificil de explicar en terminos fisicos. La idea es hacer un "truco" (el del buzo en una botella de agua) donde no es obvio porque el globo sube o baja, disfrazado de algo "sorprendente" y pedirle a la gente que complete una encuesta sobre que cree que esta pasando. Buscamos ver cuanto inventa la gente y con que sesgo en funcion de su formacion profesional. 

Los datos de la encuesta son levantados automaticamente mediante el sotware sdaps que genera los formularios y despues mediante escaneo los procesa en bloque sin necesidad de pasar los datos a manos a una compu. Del programa se puede exportar un csv con los datos en bruto. Donde a cada entrada le corresponde una encuesta completada. 

In [4]:
%load_ext autoreload
%autoreload 2
import pandas
import sys
sys.path.append('./../')
from funciones import interprete, interpretador
from IPython.display import display

En el siguiente bloque cargamos los datos y los procesamos para que sean mas legibles. Para eso hay que interpretar el csv donde viene guardados. El [csv](./DataValidada_PilotoKonex.csv) se genero automaticamente por el spdaps a partir de escanear el [archivo](./questionnaire.pdf), donde como se puede ver, hay secciones, preguntas y opciones multiples. Cada seccion esta indicada en el encabezado por un primer numero, cada preguntas por un segundo numero y cada opcion por un tercero unidos con un _ luego cada fila representa una entrada donde un 1 representa seleccionado en el caso de las opciones multiples o el valor seleccionado en caso de que se tratara de una escala. 

In [5]:
data = pandas.read_csv('./DataValidada_PilotoKonex.csv')
inter = interprete('Interprete.txt')
data = data.rename(columns={'global_id': '0_1', 'questionnaire_id': '0_2'})
#ata.drop(data.columns[[0, 1,]], axis=1, inplace=True) # Son columnas que no tienen informacion valida
data = interpretador (data,inter)


In [6]:
%matplotlib inline
display (data.head())
inter

,questionnaire_id,global_id,Nivel_seguridad,Formacion,Justificacion,Truco
0,None,None,4,[Biomedicas],[Telequinesis],[Si]
1,None,None,10,[Ciencias Sociales],[No lo se],[No se]
2,None,None,0,[],[],[]
3,None,None,7,[Artes],[Magnetismo],[No]
4,None,None,8,[Artes],[Presion],[No se]


{1: {'Titulo': 'IDs',
  1: {'Pregunta': 'global_id'},
  2: {'Pregunta': 'questionnaire_id'}},
 2: {'Titulo': 'Formacion',
  1: {'Pregunta': 'Formacion',
   1: {'Opcion': 'Artes'},
   2: {'Opcion': 'Biomedicas'},
   3: {'Opcion': 'Economicas'},
   4: {'Opcion': 'Ciencias Exactas'},
   5: {'Opcion': 'Fisica'},
   6: {'Opcion': 'Ciencias Sociales'},
   7: {'Opcion': 'Derecho'},
   8: {'Opcion': 'Otra Carrera'},
   9: {'Opcion': 'Psicologia'},
   10: {'Opcion': 'Ninguna'},
   11: {'Opcion': 'Otro'}}},
 3: {'Titulo': 'Ver',
  1: {'Pregunta': 'Justificacion',
   1: {'Opcion': 'Calor'},
   2: {'Opcion': 'Telequinesis'},
   3: {'Opcion': 'Magnetismo'},
   4: {'Opcion': 'Energia Cosmica'},
   5: {'Opcion': 'Presion'},
   6: {'Opcion': 'Fisica Cuantica'},
   7: {'Opcion': 'Electricidad'},
   8: {'Opcion': 'No lo se'}}},
 4: {'Titulo': 'Justificacion',
  1: {'Pregunta': 'Nivel_seguridad'},
  2: {'Pregunta': 'Truco',
   1: {'Opcion': 'Si'},
   2: {'Opcion': 'No'},
   3: {'Opcion': 'No se'}}}}